In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SQLContext, SparkSession

In [2]:
import time
import shutil
import sys
import requests

In [3]:
def aggregate_tags_count(new_values, total_sum):
	return sum(new_values) + (total_sum or 0)


def process_rdd(time, rdd):
	print(".:----------- %s -----------:." % str(time))
	try:
		sqlContext = SQLContext(rdd.context)
		# Get spark sql singleton context from the current context
		#sql_context = get_sql_context_instance(rdd.context)
		# convert the RDD to Row RDD
		row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
		# create a DF from the Row RDD
		hashtags_df = sqlContext.createDataFrame(row_rdd)
		# Register the dataframe as table
		# hashtags_df.registerTempTable("hashtags") # Deprecated
		hashtags_df.createOrReplaceTempView("hashtags")
		# get the top 10 hashtags from the table using SQL and print them
		hashtag_counts_df = sqlContext.sql(
		"select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
		# print(type(hashtag_counts_df))
		hashtag_counts_df.show()
		# call this method to prepare top 10 hashtags DF and send them
		# send_df_to_dashboard(hashtag_counts_df)
	except:
		e = sys.exc_info()[0]
		print("Error: %s" % e)


# def send_df_to_dashboard(hashtag_counts_df):
	# present it visually


In [4]:
checkpoint_path = r"C:\Users\AminFarvardin\Downloads\chkpointTwitter"
shutil.rmtree(checkpoint_path, ignore_errors=True)

sc = SparkContext("local[2]", "TwitterStreamApp")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 2)
# sqlContext = SQLContext(sc)
ssc.checkpoint(checkpoint_path)

dataStream = ssc.socketTextStream("127.0.0.1", 9009)

words = dataStream.flatMap(lambda line: line.split(" "))
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# tags_totals.pprint()
tags_totals.foreachRDD(process_rdd)

In [ ]:
ssc.start()

+-----------------+-------------+
|          hashtag|hashtag_count|
+-----------------+-------------+
|  #IRGCterrorists|           16|
|      #مهسا_امینی|           13|
|#IRGCterroristsRT|            9|
|  #من_وکالت_میدهم|            4|
|    #آرمیتا_عباسی|            3|
|   #توماج_صالحیRT|            2|
|     #RezaPahlavi|            2|
|     #توماج_صالحی|            2|
|          #IRG…RT|            1|
|        #مهسا_…RT|            1|
+-----------------+-------------+

.:----------- 2023-01-23 12:32:50 -----------:.
<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------+-------------+
|          hashtag|hashtag_count|
+-----------------+-------------+
|  #IRGCterrorists|           21|
|      #مهسا_امینی|           17|
|#IRGCterroristsRT|           10|
|    #آرمیتا_عباسی|            4|
|  #من_وکالت_میدهم|            4|
|     #توماج_صالحی|            4|
|     #RezaPahlavi|            3|
|        #مهسا_…RT|            2|
|   #توماج_صالحیRT|            2|
|   #IRGCterror…RT|      

In [6]:
ssc.stop()

Error: <class 'py4j.protocol.Py4JJavaError'>
